In [12]:
import os
import shutil
from glob import glob
import cv2

def rearrange_files(source_folder, destination_folder):
    for root, dirs, files in os.walk(source_folder):
        relative_path = os.path.relpath(root, source_folder)
        destination_path = os.path.join(destination_folder, relative_path)
        os.makedirs(destination_path, exist_ok=True)

def move_files(source_folder):
    destination_folder = source_folder
    for root, dirs, files in os.walk(source_folder):
        relative_path = os.path.relpath(root, source_folder)
        destination_path = os.path.join(destination_folder, relative_path)
        os.makedirs(destination_path, exist_ok=True)
        for file in files:
            source_file = os.path.join(root, file)
            destination_file = os.path.join(destination_path, file)
            shutil.move(source_file, destination_file)
            print(f'Moved {source_file} to {destination_file}')
            
def move_files(source_folder, destination_folder):
    for root, dirs, files in os.walk(source_folder):
        relative_path = os.path.relpath(root, source_folder)
        destination_path = os.path.join(destination_folder, relative_path)
        os.makedirs(destination_path, exist_ok=True)
#source_folder = 'source_folder'


In [13]:
reference_folder = "/mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/train/"
source_folder = "/mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/pix2pix_data_augmentation/large_patches/"
destination_folder = "/mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/pix2pix_data_augmentation_rearranged/"


In [14]:
# #add information about what pix2pix model was used to generate the images (one model for each class)
# for root, dirs, files in os.walk(source_folder):
#     relative_path = os.path.relpath(root, source_folder)
#     for file in files:
#         # print(os.path.join(root, file), os.path.join(root, 'pix2pix_' + root.split('/')[-2] + '_' +  file))
#         shutil.move(os.path.join(root, file), os.path.join(root, 'pix2pix_' + root.split('/')[-2] + '_' +  file))

In [15]:
rearrange_files(reference_folder, destination_folder)

In [16]:
images = sorted(glob(os.path.join(source_folder, "**/*img*"), recursive=True)) #sorted(glob(os.path.join(datadir, "**/*img.jpg"), recursive=True))
masks = sorted(glob(os.path.join(source_folder, "**/*mask*"), recursive = True)) #sorted(glob(os.path.join(datadir, "**/*mask.jpg"), recursive = True))

In [17]:
len(masks)

4774

In [18]:
#remove all elements that are not files (directories)
images = [x for x in images if os.path.isfile(x)]
masks = [x for x in masks if os.path.isfile(x)]

In [19]:
img_case_id_folders = [root for root, dirs, files in os.walk(destination_folder) if 'img' in root]
mask_case_id_folders = [root for root, dirs, files in os.walk(destination_folder) if 'mask' in root]

In [20]:
id_cases = [f.split('/')[-2] for f in img_case_id_folders]

In [21]:
len(images)

4773

In [25]:
# c = 0
# for img in images:
#     #rename the file replacing '-' with '_'
#     os.rename(img, img.replace('-', '_'))
#     img_name = img.split('/')[-1].replace('-', '_')
#     to_copy = [x for x in id_cases if x + '_' in img_name] 
#     if len(to_copy) > 1:
#         raise Exception (f'Error: {img_name} in more than one folder')
#     else: 
#         #retrive folder of the case that is in to_copy
#         folder = [x for x in img_case_id_folders if to_copy[0] in x][0]
#         # print(f'copy {img} to {folder}')
#         shutil.copy(img.replace('-', '_'), os.path.join(folder, img_name))
#         c = c + 1
# print(f'Number of images copied: {c}')

c = 0

for mask in masks:
    try:
        os.rename(mask, mask.replace('-', '_'))
    except:
        print(f'Already renamed {mask}')
    #check if name file is in the format X-X instead of X_X
    mask_name = mask.split('/')[-1].replace('-', '_')
    to_copy = [x for x in id_cases if x + '_' in mask_name]
    if len(to_copy) > 1:
        raise Exception (f'Error: {mask_name} in more than one folder')
    else:
        folder = [x for x in mask_case_id_folders if to_copy[0] in x][0]
        #if mask has 3 channels, convert to 1 channel
        if cv2.imread(mask.replace('-', '_')).shape[-1] == 3:
            mask_array = cv2.imread(mask.replace('-', '_'))
            mask_array = cv2.cvtColor(mask_array, cv2.COLOR_RGB2GRAY)
            #assert that data mask array shape is 2048x2048 with 1 channel
            assert mask_array.shape == (2048, 2048), f'Error: mask array shape is {mask_array.shape} instead of (2048,2048)'
            cv2.imwrite(os.path.join(folder, mask_name), mask_array)
        else:
            shutil.copy(mask.replace('-', '_'), folder)
        c = c + 1
print(f'Number of masks copied: {c}')

Alrady renamed /mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/pix2pix_data_augmentation/large_patches/56Nx/mask/pix2pix_56Nx_11-356_111_11264_5120_mask.png
Number of masks copied: 4773
